# Summary

Start tying openai and youtube functionality together to manage the punctuation process.

In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound

from jabberwocky.config import C
from jabberwocky.openai_utils import query_gpt3
from jabberwocky.utils import load_prompt
from jabberwocky.youtube import realign_punctuated_text, get_transcripts, \
    text_segment, video_id
from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [83]:
class UnpunctuatedTranscript:
    
    def __init__(self, df_gen, **kwargs):
        self.df_gen = df_gen
        self.df_punct = self.df_gen.copy()
        self.df_punct['text'] = np.nan
        # TODO: adjust so prompt template saved separately from prompt_kwargs.
        # TODO: adjust load_prompt to have option to skip .format call 
        # if no text passed in.
        self.prompt_fmt = load_prompt('punctuate', **kwargs)
        
    @property
    def df(self):
        # TODO: maybe should return self.df_punct, possibly after punct query? 
        # Unsure.
        return self.df_gen
    
    # Not sure yet about this interface.
#     def _punctuate_chunk(self, ...):
#         pass

In [81]:
class PunctuatedTranscript:
    
    def __init__(self, df_gen, df_punct, **kwargs):
        """
        kwargs:
            Just for compatibility with UnpunctuatedTranscript, which needs
            these to specify args like 'rstrip' when loading a prompt.
        """
        self.df_gen = df_gen
        self.df_punct = df_punct
        
    @property
    def df(self):
        return self.df_punct

In [78]:
@auto_repr
class Transcript:
    
    def __init__(self, url, **kwargs):
        self.url = url
        self.id = video_id(url)
        self._transcript = self._fetch_transcripts(url, **kwargs)
        self.is_generated = isinstance(self._transcript,
                                       UnpunctuatedTranscript)
        self.start_time, self.end_time = self.df.start.ends(1)
            
    def time_range(self, start, end, full_sentences=True):
        assert end > start, 'End time must be later than start time.'
        assert start > 0 and end > 0, 'Times must be non-negative.'

        df = self.df
        if start < self.start_time:
            start_idx = 0
        else:
            start_idx = df.loc[df.start <= start].index[-1]

        if end > df.start.iloc[-1]:
            end_idx = df.tail(1).index[0]
        else:
            end_idx = df.loc[df.start >= end].index[0]
        return df.iloc[start_idx:end_idx+1]
#         return ' '.join(df.iloc[start_idx:end_idx+1].text)
    
    @property
    def df(self):
        return self._transcript.df
    
    def _fetch_transcripts(self, url, **kwargs):
        """Wrapper to fetch youtube transcripts and create the appropriate
        transcript object depending on whether a manually generated (i.e.
        punctuated) transcript was retrieved.
        
        Parameters
        ----------
        url: str
        verbose: bool
        """
        df_gen, df_man, _ = self.get_transcripts(
            url, verbose=kwargs.get('verbose', True)
        )
        if df_man is None:
            return UnpunctuatedTranscript(df_gen, **kwargs)
        else:
            return PunctuatedTranscript(df_gen, df_man, **kwargs)
        
    @staticmethod
    def get_transcripts(url, verbose=True):
        """Fetch one or more transcripts for a youtube video given its URL.

        Parameters
        ----------
        url: str
            Don't include any channel-related suffix. E.G. use
            https://www.youtube.com/watch?v=OZbCRN3C_Hs, not
            https://www.youtube.com/watch?v=OZbCRN3C_Hs&ab_channel=BBC.
        verbose: bool
            Warn

        Returns
        -------
        DotDict: Contains keys 'id' (maps to video ID str), 'generated',
        and 'manual' (the latter two lap to pandas dfs or None if no 
        transcript was found). Manual transcripts are human-created.
        Generated transcripts are a bit lower quality and tend to lack
        punctuation.
        """
        langs = ['en', 'en-GB']
        id_ = video_id(url)
        res = {'generated': None, 'manual': None}
        trans_list = YouTubeTranscriptApi.list_transcripts(id_)
        res['generated'] = trans_list.find_generated_transcript(langs)
        try:
            res['manual'] = trans_list.find_manually_created_transcript(langs)
        except NoTranscriptFound:
            if verbose: warnings.warn('No manual transcript found.')
        if verbose:
            non_eng = [k for k, v in res.items()
                       if v and ('United Kingdom' in v.language)]
            if non_eng:
                warnings.warn(
                    f'{non_eng} {"has" if len(non_eng) == 1 else "have"} '
                    'language en-GB, not en.'
                )
        return Args(**{k: pd.DataFrame(v.fetch()) if v else v 
                       for k, v in res.items()},
                    id=id_)
    
    def __str__(self):
        return f'{type(self).__name__}(url={self.url}, '\
               f'is_generated={self.is_generated})'

In [79]:
url = 'https://www.youtube.com/watch?v=AtTsn1Ia4JY&ab_channel=LukeThomas'
trans = Transcript(url)
trans

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: UserWarning: No manual transcript found.


You should probably adjust max_tokens based on the length of the input. Bumping up to engine 2 or 3 might help a little, but engine 1 is serviceable (probably best to avoid 0 though). You should probably try training a huggingface model to add punctuation instead of using gpt3 credits though.
-------------------------------------------------------------------------------



Transcript(url='https://www.youtube.com/watch?v=AtTsn1Ia4JY&ab_channel=LukeThomas')

In [80]:
trans.time_range(7, 17)

,text,start,duration
4,absolutely spectacular fashion knocking,6.879,5.521
5,out dan hooker in just over a couple of,10.160,2.800
6,minutes,12.400,3.359
7,at ufc 257 in the co-main event you want,12.960,5.520
8,to talk about ufc debuts wow but,15.759,4.561
9,how did he do it yes of course a big,18.480,3.360


In [82]:
trans._transcript.prompt_fmt

{'engine_i': 1,
 'prompt': "Please add punctuation to these passages so that they are grammatically correct:\n\nPassage: thank you I'm honored to be with you today for your commencement from one of the finest universities in the world truth be told I never graduated from college and this is the closest I've ever gotten to a college graduation today I want to tell you three stories from my life that's it no big deal just three stories the first story is about connecting the dots I dropped out of Reed College after the first six months but then stayed around as a drop-in for another 18 months or so before I really quit so why did I drop out it started before I was born my\n\nPassage with punctuation: Thank You. I am honored to be with you today at your commencement from one of the finest universities in the world. Truth be told, I never graduated from college and this is the closest I've ever gotten to a college graduation. Today I want to tell you three stories from my life. That's it. 